## Autoencoders
**Author:** [westny](https://github.com/westny) <br>
**Date created:** 2021-05-04 <br>
**Last modified:** 2021-05-04 <br>
**Description:** Implementation Autoencoders using different strategies, including a base version (used in paper), a BNAE and Sparse AE. <br>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Encoder modules

### Base autoencoder

In [ ]:
class AutoEncoder(nn.Module):
    """ Encoder """
    def __init__(self, n_features=36, hidden_size=20, n_layers=1, dropout_prob=0.35):
        super(AutoEncoder, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(n_features, hidden_size, n_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, hidden_size)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'))
    
    def forward(self, x):
        x = x[:, :, 4:].clone()
        hc = self.init_hidden(x.size(0))
        _, (hidden, cell) = self.lstm(x, hc)
        hidden = hidden[-1,:,:]
        hidden = self.fc(hidden)
        return hidden

### Autoencoder with batchnorm layer after output

In [ ]:
class AutoEncoderBN(nn.Module):
    """ Encoder """
    def __init__(self, n_features=36, hidden_size=20, n_layers=1, dropout_prob=0.35):
        super(AutoEncoderBN, self).__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(n_features, hidden_size, n_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, hidden_size)
        
        self.bn = nn.BatchNorm1d(hidden_size)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'))
    
    def forward(self, x):
        x = x[:, :, 4:].clone()
        hc = self.init_hidden(x.size(0))
        _, (hidden, cell) = self.lstm(x, hc)
        hidden = hidden[-1,:,:]
        hidden = self.fc(hidden)
        hidden = self.bn(hidden)
        return hidden

### Sparse encoder

In [ ]:
class AutoEncoderSparse(nn.Module):
    """ Should be used together with appropriate regularization, 
        e.g L1 or KL on the output."""
    def __init__(self, n_features=36, hidden_size=20, n_layers=1, dropout_prob=0.35):
        super(AutoEncoderSparse, self).__init__()
        self.n_features = n_features
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(n_features, hidden_size, n_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, hidden_size)

    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'))
    
    def forward(self, x):
        x = x[:, :, 4:].clone()
        hc = self.init_hidden(x.size(0))
        _, (hidden, cell) = self.lstm(x, hc)
        hidden = hidden[-1,:,:]
        hidden = self.fc(hidden)
        return F.relu(hidden)

### Decoder

In [ ]:
class Decoder(nn.Module):
    """ Decoder """
    def __init__(self, n_features=20, output_size=36, n_layers=1, seq_len=20, dropout_prob=0.25):
        super(Decoder, self).__init__()
        
        self.seq_len = seq_len
        self.hidden_size = 2 * n_features
        self.n_layers = n_layers
        
        self.lstm = nn.LSTM(n_features,
                            self.hidden_size,
                            n_layers,
                            batch_first=True)
        
        self.fc = nn.Linear(self.hidden_size, output_size)

    
    def init_hidden(self, batch_size=1):
        return (torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'),
                torch.zeros(self.n_layers, batch_size, self.hidden_size, device='cuda'))
    
    def forward(self, x):
        x = x.unsqueeze(1).repeat(1, self.seq_len, 1)
        hc = self.init_hidden(x.size(0))
        
        x, (hidden_state, cell_state) = self.lstm(x, hc)

        x = x.reshape((-1, self.seq_len, self.hidden_size))
       
        out = self.fc(x)
        return out

### Combined

In [ ]:
class AutoEncoderDecoder(nn.Module):
    """ Encoder """
    def __init__(self, embedding_size=128):
        super(AutoEncoderDecoder, self).__init__()
        self.embedding = embedding_size
        self.encoder = AutoEncoder(hidden_size=self.embedding)
        self.decoder = Decoder(n_features=self.embedding)
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded